# DATE 2025 (W07.4)
# Pushing TinyML Forward: End-to-end Near-Memory RISC-V Computing
---

<img src="assets/logo.png" alt="MATCH logo" width="1000"/>

---
**MATCH** (**M**odel-**A**ware **T**VM-based **C**ompiler for **H**eterogeneous hardware) is a DNN compiler that exploits [Apache TVM](https://tvm.apache.org/)'s BYOC framework, targeting the optimized deployment of DNN applications onto heterogeneous edge System-on-Chip (SoC) platforms.

MATCH is designed to make the process of supporting new targets as simple as possible, thanks to a compilation process guided by _high-level model-based hardware abstractions_. 

The rest of this notebook shows how to extend MATCH to support the deployment of simple DNNs on the _**ARCANE**_ SoC.


## Imports

Set the required paths and import needed classes and functions from MATCH:

In [ ]:
import os
import sys
import match
from match.target.target import MatchTarget
from match.target.memory_inst import MemoryInst
from match.transform.requant import MatchRequantRewriter
from match.target.exec_module import ExecModule, ModuleLib
from match.partition.partitioning_pattern import PartitioningPattern
from match.target.exec_module import ComputationalApis
from match.node.node import MatchNode
from match.utils.utils import get_default_inputs
from match.model.model import MatchModel
from tvm.relay.dataflow_pattern import wildcard, is_op, is_constant
from tvm import relay
import onnx

from utils import plot_results
import numpy as np
import matplotlib.pyplot as plt

CURR_PATH = os.getcwd()

## Defining a new target in MATCH

The first step to support a new SoC in MATCH is to define:
- The entire SoC as a subclass of the generic `MatchTarget` class.
- Each accelerator (only one in this case, i.e., Arcane) as a subclass of `ExecModule`.


### Defining an Accelerator (`ExecModule`)

The code below contains the definition of the Arcane class:
- The constructor defines the accelerator name and points to the paths of any required support library that should be compiled together with the generated inference code.
- The `partitioning_patterns` method defines DNN graph patterns that this accelerator supports, each with optional constraints on geometry, data format, etc. In this example, for simplicity, we support `dense` operations with `int32` format, but in general we can match any number of patterns of arbitrary complexity.
- The `comp_apis_def` method defines the APIs of the accelerator (in this case, only the computation APIs, but in general we also support APIs for accelerator setup, data transfer, etc). Essentially, for each pattern supported by the accelerator, it points to backend functions to execute the corresponding operation.
- The `update_constants` method is overridden to modify the default data layout for constant tensors (i.e., weights) of TVM. This is needed because Arcane expects the weights of dense layers to be stored in the CN layout(input channels, output channels) instead of the default (NC).
- The `include_list` method defines extra include paths for compilation.
- If necessary for DSE, you can also override the `zigzag_cost_model` to return an instance of `match.cost_model.zigzag.ZigZagMatchCostModel` as detailed in the documentation.


In [ ]:
class Arcane(ExecModule):
    def __init__(self):
        super().__init__(
            name = "arcane",
            libs_required = {
                "arcane_helper": ModuleLib(name="arcane_helper", base_path=CURR_PATH+"/arcane_helper"),
            },
        )
    
    def partitioning_patterns(self):
        def dense_pt():
            """Create pattern for matmul."""
            dense = is_op("nn.dense")(
                wildcard(), is_constant()
            )
            return dense
        def only_int32_dense(node):
            dense = node
            if dense.checked_type.dtype != "int32":
                return False
            return True
        return [
            PartitioningPattern(
                name="DENSE_PT",
                pattern=dense_pt,
                additional_checks=only_int32_dense
            ),
        ]

    def comp_apis_def(self, computational_apis: ComputationalApis=None, pattern_name = "dense"):
        computational_apis.compute_tile = "arcane_compute_wrapper"
        return computational_apis
    
    def update_constants(self, match_node: MatchNode=None, pattern_name: str="dense"):
        for w_tensor in match_node.const_tensors.values():
            if "dense" in w_tensor.name:
                if w_tensor.layout!="CN":
                    w_tensor.data = w_tensor.data.transpose(1,0)
                    w_tensor.dims = [w_tensor.dims[1], w_tensor.dims[0]]
                w_tensor.layout = "CN"
                
    def include_list(self):
        return ["arcane_helper/arcane_helper"]

    # def zigzag_cost_model(self):
    #     return ExampleCostModel

### Defining a Full SoC (`MatchTarget`)

Next, we define the entire SoC, which in this case is based on the [X-Heep](https://github.com/esl-epfl/x-heep) platform, as a `MatchTarget`. In this case, the constructor defines:
- The constructor defines SoC name and the list of its accelerators (`exec_modules`), as well as various paths and function names for compilation.
- The `network_transformations` methods defines a set of custom DNN graph transformation passes to be executed when targeting this SoC. In our case, we use the built-in `MatchRequantRewriter` to replace demanding division operations during requantization with right shifts. Any number of transformations can be defined.
- The `host_memories` method defines the memory hierarchy of the target (visible to TVM), which in this case contains a single-level L2 memory composed of 1024 32-bit words.

In [ ]:
ARCANE_L2_MEMORY_SIZE = 8*32*1024

class XHeepSoC(MatchTarget):
    def __init__(self):
        super().__init__(
            exec_modules = [
                Arcane()
            ]
            , name = "xheepsoc",
        )
        self.makefile_path = CURR_PATH+"/arcane_lib/Makefile"
        self.tvm_runtime_include_path = CURR_PATH+"/arcane_lib/tvm_runtime.h"
        self.tvm_runtime_src_path = CURR_PATH+"/arcane_lib/tvm_runtime.c"
        self.init_funcs = ["arcane_helper_init_l1_mem"]
        self.include_list = ["arcane_helper/arcane_helper"]

    def network_transformations(self, opts):
        return [
            ("requant", MatchRequantRewriter()),
        ]

    def host_memories(self):
        return [
            MemoryInst(name="ARCANE_L2_MEM", k_bytes=ARCANE_L2_MEMORY_SIZE),
        ]

## The `arcane_helper` Plugin Library
MATCH's template-based compilation requires each accelerator to provide a backend library of elementary inference operations (usually in C/C++). This library clearly depends on the target, and is typically provided by the HW manufacturers. 


In our simple example, we just need to define two functions:
- To initialize the accelerator
- To perform computation (of a dense layer).
Additional functions can be defined to support more DNN operations.

```c
void arcane_helper_init_l1_mem(){
    l1_hal_init(0, l1_loader, sizeof(l1_loader), 1);
}

void arcane_compute_wrapper(MatchCtx* ctx){
    MatchTensor* tensors = ctx->tensors->tensors;
    int num_tensors = ctx->tensors->num_tensors;
    int out_chs = tensors[num_tensors-1].tiles[1].size;
    int inp_chs = tensors[0].tiles[1].size;
    // reserve matrix in the ARCANE NMC module for the activations
    xmr(m0, tensors[0].base_pt, 1, inp_chs, 1, 1);
    // reserve matrix in the ARCANE NMC module for weights stored in CN format
    xmr(m1, tensors[1].base_pt, inp_chs, out_chs, 1, 1);
    // reserve matrix in the ARCANE NMC module for the outputs
    xmr(m2, tensors[num_tensors-1].base_pt, 1, out_chs, 1, 1);
    // dense op
    carus_mmul_tiling(m2, m0, m1, mNONE, 0, 0);
}
```

## Compiling the DNN

Now that MATCH has been extended to support our SoC, we can compile full DNNs with it. In this example, we use an `int32` ONNX model of a MLP network for MNIST digit recognition. This ONNX can be generated with any ONNX-compilant framework. In our case, we used [PLiNIO](https://github.com/eml-eda/plinio/).

You can view the network's graph by dragging and dropping the `.onnx` file in [Netron](https://netron.app).

To compile it, we define a `MatchModel` as follows, where:
- `default_inputs` is an optional parameter used to add a (properly converted) input to the generated code, for testing purposes.
- `handle_out_fn` is the name of an optional function used to post-process the network output (in our case, determine the predicted class from the DNN scores.

In [ ]:
onnx_file_path = CURR_PATH+"/model/mnist.onnx"
input_path = f"{CURR_PATH}/model/input.txt"

mnist_model = MatchModel(
    filename=onnx_file_path,
    model_name="mnist",
    default_inputs=get_default_inputs(
        mod=relay.frontend.from_onnx(onnx.load(onnx_file_path))[0],
        input_files=[input_path],
    ),
    handle_out_fn="mnist_handle_output",
)

### Starting the Compilation

The compilation can be ran with the following code:


In [ ]:
match.match(
    model=mnist_model,
    target=XHeepSoC(),
    output_path=CURR_PATH+"/output",
)

##  Generated Code

The generated code is split over multiple files. 
- `./output/src/main.c` is the entry point, which essentially calls:
    * The runtime function that runs the inference (`match_mnist_runtime`).
    * The custom output handler function to interpret the results (`match_handle_output`).

The runtime function (defined in `./output/src/mnist/runtime.c`), in turn, calls functions define in the `codegen` folder, which contains the most interesting part of the DNN-generated code:

In [ ]:
!ls output/codegen/mnist/src/

The first two files in this folder (`_lib0.c` and `_lib1.c`) contain TVM-generated functions, including constant allocations, lowered code for unsupported operations, and the main inference function. The remaining files (`_lib2.c` etc) contain BYOC-generated code. In our case, this corresponds to the functions implementing the matched dense layers.

#### TVM-Generated Code

In [ ]:
!tail -n 18 output/codegen/mnist/src/mnist_lib1.c

#### MATCH-generated Code

In [ ]:
!cat output/codegen/mnist/src/mnist_lib2.c

## Host-only Compilation

MATCH allows you to easily disable code generation for accelerators, thus generating a host-only version of the code, which can be useful to measure acceleration.

In [ ]:
arcane_cpu_only = XHeepSoC()
arcane_cpu_only.disable_exec_module("arcane")
match.match(
    model=mnist_model,
    target=arcane_cpu_only,
    output_path=CURR_PATH+"/output_cpu",
)

In this case, the codegen folder only contains TVM files, which also include lowered versions of dense layers:

In [ ]:
!ls ./output_cpu/codegen/mnist/src/

In [ ]:
!cat ./output_cpu/codegen/mnist/src/mnist_lib1.c

## Analyzing Relay IR

We can also compare the TVM Relay IR graphs generated with/without support for the Arcane accelerator.

In the first case, we offload dense operations to ARCANE exploiting MATCH:

In [ ]:
!cat output/models/mnist/relay/mnist.relay

While in the CPU-only version everything is handled by the host:

In [ ]:
!cat output_cpu/models/mnist/relay/mnist.relay

If you are interested, the `models/mnist/relay` folder contains the evolution of the graph throughout the compilation. For instance, we can see that in the final one (`mnist.relay`)e `right_shift` operations are used instead of divisions.

## Testing on the Hardware

To generate the final binary for the target, you need to go into the `output` (or `output_cpu`) folders, from a Docker terminal and compile the generated C code. This step requires a target-specific C compilation environment, and will be skipped for sake of brevity.

Let's flash a pre-compiled the board and see if the model works correclty. For reference, the sample input we compiled together with the network is the following:

In [ ]:
x = np.loadtxt(input_path).reshape([28, 28])
plt.imshow(x)

## Acceleration Results

We profiled the accelerated and the host-only code and we obtained the following results:

In [ ]:
plot_results()